In [1]:
from sklearn.metrics import accuracy_score, f1_score
from EDA.DataReader import DataReader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import torch 
from torcheval.metrics.functional import multiclass_f1_score
import warnings

In [2]:
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

In [3]:
def custom_tokenizer(text):
    return text.split()

In [4]:
# pre-processing data
dataset = DataReader("UIT_VFSC") # UIT
# dataset = DataReader("") # dataset foody_raw
df_train = dataset.df_train
df_test = dataset.df_test
df_total =dataset.df_total
n_labels = int(df_total["label"].max().item() + 1)

In [5]:
# Combine with sklearn svc
tf_idf = TfidfVectorizer(tokenizer=custom_tokenizer, stop_words=[])
data_train = tf_idf.fit_transform(df_train["corpus"])
svc = SVC(kernel='linear', C=1.0, gamma=0.1)
svc.fit(data_train, df_train["label"])
data_test = tf_idf.transform(df_test["corpus"])
y_pred = svc.predict(data_test)
accuracy_svc = accuracy_score(df_test["label"], y_pred)
print(f"Accuracy: {accuracy_svc:.4f}")

y_pred = torch.tensor(y_pred).type(torch.long)
label = torch.tensor(df_test["label"].tolist()).type(torch.long)
mf1 = multiclass_f1_score(y_pred, label, num_classes=n_labels, average='macro')
wf1 = multiclass_f1_score(y_pred, label, num_classes=n_labels, average='weighted')
print(f"F1-Score: {max(mf1, wf1):.4f}")

Accuracy: 0.8866
F1-Score: 0.8697
